# Zindi - Challenge

https://zindi.africa/competitions/predict-the-global-spread-of-covid-19

April 9, 2020

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt 
import requests
import lxml.html as lh

## Import Data

In [2]:
#Read train 
deaths = pd.read_csv('data/deaths.csv')
cases = pd.read_csv('data/cases.csv')
deaths.drop(['Province/State'], axis=True)
deaths.shape

(274, 84)

## Add variables


### Temperature 

In [3]:
def web_scraping_data(link):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(link)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')

    tr_elements = doc.xpath('//tr')
    col=[]
    i=0
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        #print ('%d:"%s"'%(i,name))
        col.append((name,[]))

    for j in range(1,len(tr_elements)):
        T=tr_elements[j]
        i=0

        for t in T.iterchildren():
            data=t.text_content() 
            if i>0:
                try:
                    data=int(data)
                except:
                    pass

            col[i][1].append(data)
            i+=1

    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    
    return df

In [4]:
code_country = web_scraping_data("https://www.iban.com/country-codes")

In [5]:
code_country.shape

(249, 4)

Source 
- https://datacatalog.worldbank.org/dataset/climate-change-knowledge-portal-historical-data

In [6]:
temp = pd.read_csv('variable/temp.csv')

In [7]:
temp.shape

(178, 14)

In [8]:
temperature = pd.merge(left=temp, right=code_country, left_on='ISO_3DIGIT', right_on='Alpha-3 code')

In [9]:
temperature = temperature.drop(['Numeric','Alpha-2 code'], axis=True)

In [10]:
death1 = pd.merge(left=deaths, right=temperature, left_on='Territory', right_on='Country')

In [11]:
death1.shape

(229, 100)

I am loosing 45 countries which are 

In [12]:
l1 = list(death1.Country)
l2 = list(deaths.Territory)
[x for x in l2 if x not in l1]

['Andorra',
 'Antigua and Barbuda',
 'Bahrain',
 'Barbados',
 'Cabo Verde',
 'Democratic Republic of the Congo (the)',
 'Faroe Islands',
 'Greenland',
 'French Polynesia',
 'Mayotte',
 'Reunion',
 'Saint Barthelemy',
 'St Martin',
 "Democratic People's Republic of Korea (the)",
 'Liechtenstein',
 'Maldives',
 'Malta',
 'Republic of Moldova (the)',
 'Monaco',
 'Aruba',
 'Curacao',
 'North Macedonia',
 'Saint Lucia',
 'San Marino',
 'Seychelles',
 'Singapore',
 'Taiwan',
 'United Republic of Tanzania (the)',
 'Bermuda',
 'Cayman Islands',
 'Montserrat',
 'Dominica',
 'Grenada',
 'Syrian Arab Republic (the)',
 'Saint Kitts and Nevis',
 'Sao Tome and Principe',
 'Kiribati',
 'Marshall Islands (the)',
 'Micronesia (Federated States of)',
 'Nauru',
 'Palau',
 'Republic of Korea (the)',
 'Samoa',
 'Tonga',
 'Tuvalu']

### Others epidemy 

SARS-CoV

China Canada, Hong Kong Special Administrative Region of China, Chinese Taipei, Singapore, and Hanoi in Viet Nam.

Ebola 

The Democratic Republic of the Congo
Gabon
Guinea
Italy
Ivory Coast
Liberia
Mali
Nigeria
Philippines
Russia
Senegal
Sierra Leone
South Africa
South Sudan
Spain
Uganda
United Kingdom
United States
What causes Ebol

Mers - https://www.kaggle.com/imdevskp/mers-outbreak-dataset-20122019

Ebola - https://www.kaggle.com/imdevskp/ebola-outbreak-20142016-complete-dataset

Sars - https://www.kaggle.com/imdevskp/sars-outbreak-2003-complete-dataset

### Age / density / wealth /sex 

http://datatopics.worldbank.org/universal-health-coverage/coronavirus/

See relevant indicators

https://datacatalog.worldbank.org/search?search_api_views_fulltext_op=AND&f%5B0%5D=field_collection_field%3A2026&sort_by=field_wbddh_modified_date

https://data.worldbank.org/indicator/en.pop.dnst

In [13]:
def wordbankdata(variable, tab):
    data = pd.read_csv('variable/'+variable+'.csv')
    data = data[['Country Code', '2018']]
    death = pd.merge(left=data, right=tab, left_on='Country Code', right_on='Alpha-3 code')
    death[variable] = death['2018']
    death.drop(['2018'], axis=True, inplace=True)
    return(death)

variable = '%+65'

In [14]:
death2 = wordbankdata(variable, death1)

In [15]:
l1 = list(death1.Country)
l2 = list(death2.Country)
[x for x in l1 if x not in l2]

['French Guiana', 'Guadeloupe']

In [16]:
variable = '15-64pop'
death2 = wordbankdata(variable, death2)
death2 = wordbankdata('density', death2)
death2 = wordbankdata( 'sex', death2)

In [17]:
rich = pd.read_csv('variable/income.csv')
rich = rich[['Country Code', 'IncomeGroup']]

In [18]:
death2 = pd.merge(left=rich, right=death2, left_on='Country Code', right_on='Alpha-3 code')

In [19]:
death2.drop(['Alpha-3 code','Country Code_y', 'Country Code_x','Country Code_y' ,'Country'], axis=1, inplace=True)


In [20]:
death2.head()

,Country Code,IncomeGroup,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,...,Aug_Temp,Sept_temp,Oct_temp,Nov_Temp,Dec_temp,Annual_temp,%+65,15-64pop,density,sex
0,AFG,Low income,NaN,Afghanistan,Afghanistan,26023100.0,33.0000,65.0000,0,0,...,23.77,19.03,12.99,7.00,2.43,12.92,2.584927,54.324898,56.937760,48.635847
1,AGO,Lower middle income,NaN,Angola,Angola,24383301.0,-11.2027,17.8739,0,0,...,19.90,22.19,23.18,22.79,22.61,21.51,2.216374,50.974702,24.713052,50.530463
2,ALB,Upper middle income,NaN,Albania,Albania,2895947.0,41.1533,20.1683,0,0,...,20.48,17.16,12.27,7.58,3.65,11.27,13.744736,68.582390,104.612263,49.063095
3,ARE,High income,NaN,United Arab Emirates,United Arab Emirates (the),NaN,24.0000,54.0000,0,0,...,33.55,31.74,28.34,24.06,20.28,26.83,1.085001,84.311490,135.609110,30.636688
4,ARG,Upper middle income,NaN,Argentina,Argentina,42669500.0,-38.4161,-63.6167,0,0,...,9.02,11.53,14.67,17.54,19.83,14.22,11.117789,64.121277,16.258510,51.237348


In [21]:
death2.shape

(227, 104)

In [22]:
deaths.shape

(274, 84)

## Tourism

In [43]:
tourism = pd.read_csv('variable/tourism.csv')

tourism.count() / len(tourism)
Lets take 2011.

In [50]:
tourism = tourism[['2011','Country Code']]

## Economical factors

https://www.kaggle.com/lewisduncan93/the-economic-freedom-index

## Immunization factor

https://www.kaggle.com/lsind18/who-immunization-coverage

## Split by continent

In [23]:
continent = pd.read_csv('variable/locations.csv')

In [24]:
continent.head()

,countriesAndTerritories,location,continent,population_year,population
0,Afghanistan,Afghanistan,Asia,2020.0,38928341.0
1,Albania,Albania,Europe,2020.0,2877800.0
2,Algeria,Algeria,Africa,2020.0,43851043.0
3,Andorra,Andorra,Europe,2020.0,77265.0
4,Angola,Angola,Africa,2020.0,32866268.0


In [25]:
continent.shape

(207, 5)

In [26]:
continent.drop(['population_year','population','countriesAndTerritories'], axis=1, inplace=True)

In [27]:
death2 = pd.merge(left=continent, right=death1, left_on='location', right_on='Country/Region')

In [28]:
death2.shape

(216, 102)

### Infected cases March 1 / March 15 / March 30

We take the number of cases every 15 days since the beginning of March. 

In [ ]:
cases = pd.read_csv('data/cases.csv')

In [ ]:
cases[['2/15/20', '3/1/20', '3/30/20', '3/30/20', 'Country/Region']]

## Commorbidity / Risk Factors

### Obesity 

https://www.kaggle.com/arttua/who-obesity-by-country-2016

In [30]:
obesity = pd.read_csv('variable/obesity.csv')
obesity['Country'] = obesity['Unnamed: 0']
obesity.drop('Unnamed: 0', axis=1, inplace=True)

In [31]:
death3 = pd.merge(left=obesity, right=death2, left_on='Country', right_on='Country/Region')
death3.shape

(200, 106)

In [32]:
death2.shape

(216, 102)

### Pollution ?
https://www.kaggle.com/kweinmeister/pm25-global-air-pollution-20102017

## Medical care

### Hospital beds

https://www.kaggle.com/hamzael1/hospital-beds-by-country

In [33]:
beds = pd.read_csv('variable/beds.csv')

In [34]:
beds.count() / len(beds)

Country Name      1.000000
Country Code      1.000000
Indicator Name    1.000000
Indicator Code    1.000000
1960              0.613636
1961              0.007576
1962              0.007576
1963              0.007576
1964              0.007576
1965              0.030303
1966              0.018939
1967              0.018939
1968              0.018939
1969              0.018939
1970              0.662879
1971              0.030303
1972              0.034091
1973              0.034091
1974              0.041667
1975              0.340909
1976              0.056818
1977              0.041667
1978              0.045455
1979              0.045455
1980              0.492424
1981              0.280303
1982              0.113636
1983              0.098485
1984              0.125000
1985              0.378788
                    ...   
1991              0.363636
1992              0.318182
1993              0.412879
1994              0.318182
1995              0.318182
1996              0.443182
1

In [35]:
beds= beds[['Country Code', '2011']]

MERGE !

### Health Systems - WHO 


In [36]:
hs = pd.read_csv('variable/Health_systems.csv')

In [37]:
hs.shape

(210, 14)

In [39]:
hs.head()

,Country_Region,Province_State,World_Bank_Name,Health_exp_pct_GDP_2016,Health_exp_public_pct_2016,Health_exp_out_of_pocket_pct_2016,Health_exp_per_capita_USD_2016,per_capita_exp_PPP_2016,External_health_exp_pct_2016,Physicians_per_1000_2009-18,Nurse_midwife_per_1000_2009-18,Specialist_surgical_per_1000_2008-18,Completeness_of_birth_reg_2009-18,Completeness_of_death_reg_2008-16
0,Afghanistan,NaN,Afghanistan,10.2,5.1,77.4,57.2,162.8,17.5,0.3,0.3,0.0,42.3,NaN
1,Albania,NaN,Albania,6.7,41.4,58.0,271.5,759.7,0.7,1.2,3.6,11.6,98.4,53.0
2,Algeria,NaN,Algeria,6.6,67.7,30.9,260.4,998.2,0.0,1.8,2.2,12.1,100.0,NaN
3,Andorra,NaN,Andorra,10.4,49.1,41.7,3834.7,4978.7,NaN,3.3,4.0,83.1,100.0,80.0
4,Angola,NaN,Angola,2.9,44.1,35.2,95.2,185.8,3.6,0.2,1.3,NaN,25.0,NaN


### Test

https://www.kaggle.com/skylord/covid19-tests-conducted-by-country

In [41]:
test = pd.read_csv('variable/test/Tests_conducted_09April2020.csv')

import all files in the folder test !

## Vaccine - clinical test
https://www.kaggle.com/panahi/covid-19-international-clinical-trials#ClinicalTrials.gov_COVID_19.csv

https://www.kaggle.com/zohrarezgui/coronavirus-clinical-trials-dataset

## Non-pharmaceutical interventions



### Lockdown 

https://www.kaggle.com/jcyzag/covid19-lockdown-dates-by-country

lockdown folder !

In [ ]:
sum(deaths.Territory == 'France')

In [ ]:
deaths[deaths.Territory == 'France']